In [2]:
import csv
import random
import numpy as np

# Import datasets, classifiers and performance metrics
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
# candidate info
candidate_count = 0
candidates = []

# ballot mete data
total_voter_count = 0
total_vote_count = 0
unique_order_count = 0

# ballot data
# map a unique order of ballots to the total count of ballots in this order
 # format: tuple(first choice,second,third choice...): count
ballots = []

ballots_size = []

with open("data/dublin-north.soi", newline="\n") as data_file:
    data = csv.reader(data_file)
    i = 0
    for row in data:
        if i == 0:
            candidate_count = int(row[0])
        elif i <= candidate_count:
            candidates.append(row[1])
        elif i == candidate_count + 1:
            total_voter_count = int(row[0])
            total_vote_count = int(row[1])
            unique_order_count = int(row[2])
        else:
            ballot = tuple(int(j) for j in row[1:])
            ballots.append([ballot, int(row[0])])
            ballots_size.append(len(ballot))
        i += 1

In [4]:
# print(candidate_count)
# for key in ballots.keys():
#     print(key, ballots[key])
#     print(len(key))
# for ballot in ballots:
#     print(ballot, ballots[ballot])
#     assert len(ballot[0]) != 0
# print(train_data[0][:-1])

In [5]:
# unroll ballots
def unroll_ballots(ballots):
    unrolled_ballots = []
    for b in range(len(ballots)):
        for i in range(ballots[b][1]):
            unrolled_ballots.append(list(ballots[b][0]))
    return unrolled_ballots

# unrolled_ballots = unroll_ballots(ballots)
# print(len(unrolled_ballots), " ballots")

In [6]:
# get train and set labels
def known_target_split(unrolled_ballots, target_length):
    known_selections = []
    target_selection = []
    for b in range(len(unrolled_ballots)):
        if len(unrolled_ballots[b]) >= target_length:
            known_selections.append(list(unrolled_ballots[b][:target_length - 1]))
            target_selection.append(unrolled_ballots[b][target_length - 1])
    return known_selections, target_selection

# known_selections, target_selection = known_target_split(unrolled_ballots, 5)
# print(len(known_selections))

In [71]:
# divide into train and test set
# X_train, X_test, y_train, y_test = train_test_split(known_selections,target_selection, test_size = 0.2, random_state=1)

In [72]:
# Create a classifier: a support vector classifier
# clf = svm.SVC(gamma=0.15)

In [73]:
# clf.fit(X_train, y_train)

In [74]:
# predicted = clf.predict(X_test)

In [75]:
# print(f"{metrics.classification_report(y_test, predicted)}\n")

In [8]:
def train_svm(known_selections, target_selection):
    clf = svm.SVC(gamma=0.15)
#     X_train, X_test, y_train, y_test = train_test_split(known_selections,target_selection, test_size = 0.2, random_state=1)    
#     clf.fit(X_train, y_train)
    clf.fit(known_selections, target_selection)
    return clf
def train_lrg(known_selections, target_selection):
    clf = LogisticRegression(penalty='l1', solver='liblinear')
    clf.fit(known_selections, target_selection)
    return clf

In [12]:
# Return an order of ballot as a tuple
def predict(ballots, target_ballot, model):
    # finish this
    
    target = model.predict_proba([target_ballot])
    print(target)
    
    improved_target_ballot = list(target_ballot)
    improved_target_ballot.append(target[0])
    
#     ##################### place holder ####################
#     temp = list(target_ballot)
#     temp.append(random.randint(1,12))
#     improved_target_ballot = tuple(temp)
#     #######################################################
    
    return improved_target_ballot

In [13]:
# this code will modify the data in "ballots". Need to rerun above data reading code to restore
# data in the dictionary.
for i in range(1, candidate_count):
    # unroll ballots
    unrolled_ballots = unroll_ballots(ballots)
    
    # split into data and labels, where the selection at i + 1 place is the label, selections before
    # are the data
    known_selections, target_selection = known_target_split(unrolled_ballots, i + 1)
    
    # train model for predicting the i+1 preference
#     model = train_svm(known_selections, target_selection)
    model = train_lrg(known_selections, target_selection)
    # update ballots with only i preference
#     keys = list(ballots.keys())
    for b in range(len(ballots)):
        ballot = ballots[b][0]
        if len(ballot) == i : #and ballots[b][0] != 0:
            improved_ballot = predict(ballots, ballot, model)
            ballots[b][0] = improved_ballot
#             if improved_ballot in ballots:
#                 ballots[improved_ballot] += ballots[ballot]
#             else:
#                 ballots[improved_ballot] = ballots[ballot]
#             del ballots[ballot]
#             ballots[ballot] = 0 # set old count to 0, should then have no impact on final vote count
# print(ballots)

[[0.06759998 0.12536625 0.0190022  0.15515286 0.02728706 0.18526371
  0.09146079 0.00774605 0.11929793 0.12583298 0.00454014 0.07145007]]
[[0.05523721 0.05941595 0.05268534 0.06211444 0.03476585 0.10306426
  0.07765815 0.01232363 0.15826369 0.21838611 0.00588518 0.16020019]]
[[0.06706886 0.11652535 0.02200131 0.14153939 0.02861901 0.17568029
  0.09117703 0.00835286 0.12584575 0.13765141 0.00477153 0.0807672 ]]
[[0.05983918 0.07407011 0.0422073  0.0810431  0.03370571 0.12345642
  0.08332259 0.01130258 0.15209402 0.19727445 0.00568008 0.13600445]]
[[0.06774891 0.14270548 0.0139711  0.18281467 0.02447176 0.20266018
  0.0907916  0.0065717  0.10570268 0.10348691 0.00405536 0.05501965]]
[[0.06358821 0.09035013 0.03307319 0.10326958 0.03205313 0.14467252
  0.08769658 0.01016782 0.14323995 0.17397464 0.00537802 0.11253622]]
[[0.06505574 0.0989268  0.02903086 0.11546943 0.03102144 0.15525651
  0.08929129 0.00957106 0.13791376 0.16188345 0.00519379 0.10138588]]
[[0.05762471 0.06651167 0.04728413

KeyboardInterrupt: 

In [66]:
# Return a dictionary mapping candidate number to score.
# Calculate candidate scores based on Borda's Rule.
def borda_rule_result(ballots,ballots_size=None,score_decay=1):
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
    candidate_scores_imp = dict.fromkeys(range(1, candidate_count+1), 0)
#     print(candidate_scores)
    for b in range(len(ballots)):
        if b in drop_list: continue
        ballot = ballots[b][0]
        count = ballots[b][1]
        dec_cnt = 1
        for i in range(candidate_count):
            if not ballots_size is None and ballots_size[b] < i:
                candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count * pow(score_decay, dec_cnt)
                candidate_scores_imp[ballot[i]] += (candidate_count - 1 - i) * count * pow(score_decay, dec_cnt)
                dec_cnt += 1
            else:
                candidate_scores[ballot[i]] += (candidate_count - 1 - i) * count
    return candidate_scores, candidate_scores_imp

In [67]:
for i in [0, 0.25, 0.5, 0.75, 1]: 
    l,limp = borda_rule_result(ballots, ballots_size,i)
    ln = sorted(l.items(), key=lambda item: item[1])
    ln.reverse()
    limpn = sorted(limp.items(), key=lambda item: item[1])
    limpn.reverse()
    print(limp)
#     print(l)
#     maxi = 0
#     can = 0
#     for c,v in l.items():
#         if v > maxi:
#             maxi = v
#             can = c
#     print(can)
        

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0}
{1: 10486.697128295898, 2: 9008.833023071289, 3: 967.7453308105469, 4: 3859.0550384521484, 5: 6391.379642486572, 6: 2157.074333190918, 7: 8673.185150146484, 8: 5789.892848968506, 9: 9795.03205871582, 10: 4842.9600830078125, 11: 4331.194648742676, 12: 5533.4864501953125}
{1: 28495.76953125, 2: 22452.42578125, 3: 3504.4765625, 4: 9639.78515625, 5: 18963.369140625, 6: 6203.4609375, 7: 22028.515625, 8: 19311.759765625, 9: 21724.38671875, 10: 10987.859375, 11: 14988.0546875, 12: 12708.94140625}
{1: 58844.37840270996, 2: 43167.58644104004, 3: 11190.226043701172, 4: 20671.77781677246, 5: 43892.419567108154, 6: 14708.761848449707, 7: 43031.6979675293, 8: 50970.189502716064, 9: 37207.16450500488, 10: 18620.666381835938, 11: 42973.0472946167, 12: 23635.41128540039}
{1: 109177, 2: 75913, 3: 34474, 4: 46472, 5: 94296, 6: 34762, 7: 77207, 8: 122811, 9: 58198, 10: 28021, 11: 116122, 12: 44082}


In [55]:
def svt(ballots, ballots_size=[],score_decay=1):
    elim = [] # eliminated candidates
    while True:
        candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
        for b in range(len(ballots)):
            if b in drop_list: continue
            ballot = ballots[b][0]
            count = ballots[b][1]
            for i in range(candidate_count):
                if ballot[i] not in elim:
                    if not ballots_size is None and ballots_size[b] < i:
                        candidate_scores[ballot[i]] += count * score_decay
                    else:
                        candidate_scores[ballot[i]] += count
                    break
        least = 0
        first = True
        for (i,v) in candidate_scores.items():
            if i not in elim:
                if first:
                    least = i
                    first = False
                elif candidate_scores[least] > v:
                    least = i
        # eliminate lowest
        print("Eliminate "+str(least))
        elim.append(least)
        # if 1 left, return as winner
        if len(elim) == candidate_count-1: return [x for x in range(1,candidate_count+1) if x not in elim][0]

In [56]:
for i in [0, 0.25, 0.5, 0.75, 1]: print(svt(ballots, ballots_size, i))

Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
Eliminate 4
Eliminate 12
Eliminate 9
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
Eliminate 4
Eliminate 12
Eliminate 9
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
Eliminate 4
Eliminate 12
Eliminate 9
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
Eliminate 4
Eliminate 12
Eliminate 9
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
Eliminate 4
Eliminate 12
Eliminate 9
10


In [53]:
def irv(ballots, ballots_size=[],score_decay=1):
    elim = [] # eliminated candidates
    while True:
        candidate_scores = dict.fromkeys(range(1, candidate_count+1), 0)
        for b in range(len(ballots)):
            if b in drop_list: continue
            ballot = ballots[b][0]
            count = ballots[b][1]
            for i in range(candidate_count):
                if ballot[i] not in elim:
                    if not ballots_size is None and ballots_size[b] < i:
                        candidate_scores[ballot[i]] += count * score_decay
                    else:
                        candidate_scores[ballot[i]] += count
                    break
        least = 0
        best = 0
        first = True
        for (i,v) in candidate_scores.items():
            if i not in elim:
                if first:
                    least = i
                    best = i
                    first = False
                elif candidate_scores[least] > v:
                    least = i
                elif candidate_scores[best] < v:
                    best = i
        # if majority, return as winner
        if candidate_scores[best] > len(ballots)/2: return best
        # otherwise eliminate lowest and continue
        print("Eliminate "+str(least))
        elim.append(least)
        

In [54]:
for i in [0, 0.25, 0.5, 0.75, 1]: print(irv(ballots, ballots_size, i))

Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
10
Eliminate 11
Eliminate 8
Eliminate 5
Eliminate 1
Eliminate 3
Eliminate 7
Eliminate 6
Eliminate 2
10


In [39]:
def copeland(ballots, ballot_size=None, score_decay=1):
    
    candidate_scores = dict.fromkeys(range(1, candidate_count+1), dict.fromkeys(range(1, candidate_count+1), 0))
    # get dic where [i][j] -> # times i beats j
    for b in range(len(ballots)):
        if b in drop_list: continue
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count):
            for j in range(i+1,candidate_count):
                # ballot[i] beats all ballot[j]
                if not ballots_size is None and ballots_size[b] < i:
                    candidate_scores[ballot[i]][ballot[j]] += count * score_decay
                else:
                    candidate_scores[ballot[i]][ballot[j]] += count
                
    
    
    candidate_pairwise_wins = dict.fromkeys(range(1, candidate_count+1), 0)
    
    # loop through all pairwise matchups
    for i in range(1, candidate_count+1):
        for j in range(i+1, candidate_count+1):
            if candidate_scores[i][j] > candidate_scores[j][i]:
                # i beats j
                candidate_pairwise_wins[i] += 1
            else: 
                candidate_pairwise_wins[j] += 1
                
    
    max = 1
    # get candidate with most pairwise wins
    for i in range(1, candidate_count+1):
        if candidate_pairwise_wins[i] > candidate_pairwise_wins[max]:
            max = i
        print(str(i) + " wins: " + str(candidate_pairwise_wins[i]))
    return max
    
    
    

In [52]:
# Update decay with exopnential...
for i in [0, 0.25, 0.5, 0.75, 1]: print(copeland(ballots, ballots_size, i))

1 wins: 4
2 wins: 7
3 wins: 1
4 wins: 8
5 wins: 3
6 wins: 9
7 wins: 5
8 wins: 2
9 wins: 10
10 wins: 11
11 wins: 0
12 wins: 6
10
1 wins: 4
2 wins: 8
3 wins: 1
4 wins: 7
5 wins: 3
6 wins: 9
7 wins: 5
8 wins: 2
9 wins: 10
10 wins: 11
11 wins: 0
12 wins: 6
10
1 wins: 4
2 wins: 9
3 wins: 0
4 wins: 7
5 wins: 3
6 wins: 8
7 wins: 5
8 wins: 2
9 wins: 10
10 wins: 11
11 wins: 1
12 wins: 6
10
1 wins: 4
2 wins: 9
3 wins: 0
4 wins: 7
5 wins: 3
6 wins: 8
7 wins: 5
8 wins: 2
9 wins: 10
10 wins: 11
11 wins: 1
12 wins: 6
10
1 wins: 4
2 wins: 9
3 wins: 0
4 wins: 5
5 wins: 3
6 wins: 8
7 wins: 7
8 wins: 2
9 wins: 10
10 wins: 11
11 wins: 1
12 wins: 6
10


In [ ]:
# from numpy import random
# import numpy as np
def reduce_ballots(ballots, ballots_size, alpha, beta):
    array = np.array(range(len(ballots)))
    random.shuffle(array)
    change_ballots = array.tolist()
    alpha_cnt = int(len(ballots) * alpha)
    for b in change_ballots[0:alpha_cnt]: # Change first alpha random ballots
        # Can we get original data set???
        # Assuming all ballots are complete...
        step = 1
        ballot = ballots[b][0]
        count = ballots[b][1]
        for i in range(candidate_count-1,0,-1):
            r = random.uniform(0,1)
            if r > beta^step:
                ballot = ballot[:-1]
                step += 1
            else:
                break
        ballots[b] = [ballot,count]
        ballots_size[b] = candidate_count - (step-1)
        
    return ballots, ballots_size
        

In [45]:
def drop_some(alpha):
    drop_list = []
    alpha_cnt = int(len(ballots) * alpha)
    print("Drop "+str(alpha_cnt)+" from "+str(total_voter_count))
    array = np.array(range(len(ballots)))
    random.shuffle(array)
    change_ballots = array.tolist()
    for b in change_ballots:
        if alpha_cnt <= 0: break
        if ballots_size[b] < candidate_count:
            drop_list.append(b)
            alpha_cnt -= ballots[b][1]
    return drop_list

In [59]:
drop_list = drop_some(0.3)
drop_list = []

Drop 5789 from 43942


In [68]:
for b,c in ballots:
    for i in range(len(b)):
        if b[i] in b[i+1:]: 
            print(b)

[12, 4, 6, 10, 2, 7, 1, 8, 5, 3, 11, 3]
[6, 4, 12, 10, 9, 7, 1, 11, 5, 1, 3, 7]
[6, 4, 12, 10, 9, 7, 1, 11, 5, 1, 3, 7]
[4, 12, 6, 10, 9, 7, 1, 2, 11, 3, 11, 3]
[4, 12, 6, 10, 9, 7, 1, 2, 11, 3, 11, 3]
[2, 10, 9, 7, 1, 5, 8, 11, 4, 3, 11, 3]
[2, 10, 9, 7, 1, 5, 8, 11, 4, 3, 11, 3]
[2, 9, 10, 7, 1, 5, 8, 11, 4, 3, 8, 11]
[2, 9, 10, 7, 1, 5, 8, 11, 4, 3, 8, 11]
[2, 10, 9, 7, 1, 5, 8, 11, 4, 3, 11, 3]
[2, 10, 9, 7, 1, 5, 8, 11, 4, 3, 11, 3]
[2, 10, 9, 7, 1, 5, 8, 11, 4, 3, 11, 3]
[2, 10, 9, 7, 1, 5, 8, 11, 4, 3, 11, 3]
[2, 9, 10, 7, 1, 5, 8, 11, 4, 3, 8, 11]
[2, 9, 10, 7, 1, 5, 8, 11, 4, 3, 8, 11]
[10, 2, 9, 7, 1, 5, 12, 11, 6, 5, 3, 3]
[10, 2, 9, 7, 1, 5, 12, 11, 6, 5, 3, 3]
[10, 2, 9, 7, 1, 5, 12, 11, 6, 5, 3, 3]
[10, 2, 9, 7, 1, 5, 12, 11, 6, 5, 3, 3]
[3, 6, 12, 4, 9, 10, 3, 8, 5, 11, 11, 3]
[3, 6, 12, 4, 9, 10, 3, 8, 5, 11, 11, 3]
[3, 6, 12, 4, 9, 10, 3, 8, 5, 11, 11, 3]
[4, 6, 12, 9, 10, 2, 7, 11, 7, 5, 3, 3]
[4, 6, 12, 9, 10, 2, 7, 11, 7, 5, 3, 3]
[12, 4, 6, 10, 2, 7, 1, 8, 5, 3, 11

[9, 6, 10, 7, 2, 1, 5, 11, 3, 12, 6, 3]
[9, 6, 10, 7, 2, 1, 5, 11, 3, 12, 6, 3]
[10, 2, 3, 9, 5, 8, 11, 5, 6, 12, 1, 3]
[10, 2, 3, 9, 5, 8, 11, 5, 6, 12, 1, 3]
[4, 12, 10, 2, 6, 8, 11, 9, 5, 4, 1, 3]
[7, 9, 12, 10, 4, 2, 1, 7, 8, 8, 5, 3]
[7, 9, 12, 10, 4, 2, 1, 7, 8, 8, 5, 3]
[7, 4, 6, 12, 10, 2, 1, 7, 8, 8, 11, 3]
[7, 4, 6, 12, 10, 2, 1, 7, 8, 8, 11, 3]
[2, 6, 9, 3, 12, 5, 8, 11, 1, 8, 11, 3]
[2, 6, 9, 3, 12, 5, 8, 11, 1, 8, 11, 3]
[2, 6, 9, 3, 12, 5, 8, 11, 1, 8, 11, 3]
[6, 12, 10, 2, 7, 4, 9, 11, 11, 5, 11, 3]
[6, 12, 10, 2, 7, 4, 9, 11, 11, 5, 11, 3]
[7, 1, 10, 8, 2, 5, 8, 11, 4, 6, 12, 3]
[4, 6, 12, 10, 7, 1, 9, 11, 5, 8, 11, 3]
[7, 5, 10, 12, 1, 2, 7, 8, 11, 11, 11, 3]
[7, 5, 10, 12, 1, 2, 7, 8, 11, 11, 11, 3]
[7, 5, 10, 12, 1, 2, 7, 8, 11, 11, 11, 3]
[9, 10, 1, 4, 7, 12, 5, 1, 5, 11, 8, 3]
[9, 10, 1, 4, 7, 12, 5, 1, 5, 11, 8, 3]
[4, 12, 6, 10, 9, 7, 1, 2, 11, 3, 11, 3]
[4, 12, 6, 10, 9, 7, 1, 2, 11, 3, 11, 3]
[2, 4, 10, 9, 12, 6, 1, 3, 8, 11, 5, 3]
[4, 9, 12, 10, 2, 6, 7, 8, 8,

[10, 12, 4, 7, 9, 2, 7, 1, 8, 11, 11, 3]
[10, 12, 4, 7, 9, 2, 7, 1, 8, 11, 11, 3]
[10, 12, 4, 2, 6, 6, 9, 11, 5, 3, 8, 1]
[7, 4, 9, 10, 1, 12, 6, 1, 8, 11, 11, 3]
[7, 4, 9, 10, 1, 12, 6, 1, 8, 11, 11, 3]
[4, 1, 10, 9, 7, 12, 6, 4, 2, 11, 8, 3]
[4, 1, 10, 2, 6, 12, 9, 7, 11, 8, 11, 3]
[1, 9, 12, 6, 4, 7, 1, 9, 11, 8, 5, 3]
[1, 9, 12, 6, 4, 7, 1, 9, 11, 8, 5, 3]
[12, 4, 2, 3, 6, 9, 9, 5, 8, 11, 7, 1]
[10, 11, 5, 2, 7, 1, 8, 4, 6, 12, 11, 3]
[10, 11, 9, 7, 1, 2, 8, 11, 11, 8, 11, 3]
[10, 11, 9, 7, 1, 2, 8, 11, 11, 8, 11, 3]
[10, 11, 9, 7, 1, 2, 8, 11, 11, 8, 11, 3]
[10, 11, 9, 7, 1, 2, 8, 11, 11, 8, 11, 3]
[1, 10, 2, 9, 5, 11, 8, 6, 4, 1, 7, 3]
[2, 9, 3, 10, 1, 8, 12, 4, 8, 11, 6, 3]
[2, 9, 3, 10, 1, 8, 12, 4, 8, 11, 6, 3]
[7, 10, 9, 1, 2, 4, 8, 11, 12, 4, 6, 3]
[4, 9, 7, 12, 1, 7, 10, 11, 8, 8, 11, 3]
[4, 9, 7, 12, 1, 7, 10, 11, 8, 8, 11, 3]
[4, 9, 7, 12, 1, 7, 10, 11, 8, 8, 11, 3]
[12, 4, 6, 11, 8, 1, 9, 11, 8, 5, 2, 3]
[12, 4, 6, 11, 8, 1, 9, 11, 8, 5, 2, 3]
[8, 9, 10, 2, 1, 5, 8, 11, 

[12, 6, 2, 4, 9, 9, 7, 8, 3, 1, 11, 5]
[6, 2, 9, 12, 4, 5, 10, 8, 11, 8, 11, 3]
[6, 2, 9, 12, 4, 5, 10, 8, 11, 8, 11, 3]
[9, 2, 7, 10, 6, 1, 12, 11, 6, 4, 8, 3]
[9, 2, 7, 10, 3, 6, 12, 11, 8, 5, 1, 3]
[5, 2, 10, 9, 3, 7, 12, 8, 12, 1, 6, 4]
[5, 2, 10, 9, 7, 1, 3, 6, 8, 11, 11, 3]
[5, 2, 10, 9, 7, 1, 3, 6, 8, 11, 11, 3]
[10, 9, 5, 4, 12, 2, 1, 8, 11, 5, 3, 3]
[10, 9, 5, 4, 12, 2, 1, 8, 11, 5, 3, 3]
[4, 6, 7, 2, 1, 10, 10, 11, 5, 8, 11, 3]
[4, 6, 7, 2, 1, 10, 10, 11, 5, 8, 11, 3]
[2, 3, 11, 9, 5, 8, 11, 4, 1, 5, 8, 3]
[2, 3, 11, 9, 5, 8, 11, 4, 1, 5, 8, 3]
[2, 3, 11, 9, 5, 8, 11, 4, 1, 5, 8, 3]
[2, 3, 11, 9, 5, 8, 11, 4, 1, 5, 8, 3]
[4, 6, 9, 1, 10, 12, 5, 8, 11, 8, 1, 3]
[4, 6, 9, 1, 10, 12, 5, 8, 11, 8, 1, 3]
[4, 6, 9, 2, 1, 12, 10, 6, 5, 3, 11, 8]
[2, 12, 6, 4, 3, 10, 9, 11, 7, 8, 11, 3]
[2, 12, 6, 4, 3, 10, 9, 11, 7, 8, 11, 3]
[2, 3, 5, 11, 8, 8, 10, 6, 4, 12, 7, 1]
[9, 2, 10, 7, 12, 4, 1, 5, 7, 8, 11, 3]
[7, 10, 9, 1, 8, 5, 2, 12, 11, 8, 11, 3]
[7, 10, 9, 1, 8, 5, 2, 12, 11, 8, 11, 

[7, 1, 2, 9, 5, 10, 8, 5, 11, 8, 11, 3]
[7, 1, 2, 9, 5, 10, 8, 5, 11, 8, 11, 3]
[10, 4, 9, 1, 2, 3, 6, 8, 11, 12, 3, 3]
[10, 4, 9, 1, 2, 3, 6, 8, 11, 12, 3, 3]
[1, 9, 3, 12, 2, 10, 8, 8, 11, 8, 11, 3]
[1, 9, 3, 12, 2, 10, 8, 8, 11, 8, 11, 3]
[1, 9, 3, 12, 2, 10, 8, 8, 11, 8, 11, 3]
[1, 9, 3, 12, 2, 10, 8, 8, 11, 8, 11, 3]
[12, 6, 4, 3, 5, 11, 10, 8, 11, 8, 11, 3]
[12, 6, 4, 3, 5, 11, 10, 8, 11, 8, 11, 3]
[12, 6, 4, 3, 5, 11, 10, 8, 11, 8, 11, 3]
[12, 6, 4, 3, 5, 11, 10, 8, 11, 8, 11, 3]
[7, 10, 3, 5, 8, 1, 12, 8, 5, 11, 11, 3]
[7, 10, 3, 5, 8, 1, 12, 8, 5, 11, 11, 3]
[7, 10, 3, 5, 8, 1, 12, 8, 5, 11, 11, 3]
[7, 10, 3, 5, 8, 1, 12, 8, 5, 11, 11, 3]
[7, 1, 10, 2, 4, 12, 8, 7, 5, 8, 11, 3]
[7, 1, 10, 2, 4, 12, 8, 7, 5, 8, 11, 3]
[12, 4, 6, 9, 2, 1, 7, 11, 9, 8, 5, 3]
[2, 10, 9, 3, 8, 5, 12, 1, 5, 8, 11, 3]
[2, 10, 9, 3, 8, 5, 12, 1, 5, 8, 11, 3]
[2, 10, 9, 3, 8, 5, 12, 1, 5, 8, 11, 3]
[4, 10, 8, 9, 5, 3, 1, 8, 11, 5, 11, 3]
[4, 10, 8, 9, 5, 3, 1, 8, 11, 5, 11, 3]
[4, 10, 8, 9, 5, 3, 1, 8,

...

[9, 2, 6, 12, 1, 4, 4, 8, 11, 8, 11, 3]
[9, 2, 6, 12, 1, 4, 4, 8, 11, 8, 11, 3]
[6, 4, 7, 3, 10, 10, 12, 8, 8, 8, 11, 3]
[6, 4, 7, 3, 10, 10, 12, 8, 8, 8, 11, 3]
[6, 4, 7, 3, 10, 10, 12, 8, 8, 8, 11, 3]
[6, 4, 7, 3, 10, 10, 12, 8, 8, 8, 11, 3]
[12, 9, 6, 10, 7, 4, 2, 1, 8, 11, 3, 3]
[9, 7, 10, 1, 2, 5, 8, 6, 4, 12, 3, 3]
[9, 5, 10, 8, 3, 1, 12, 5, 4, 8, 11, 3]
[9, 5, 10, 8, 3, 1, 12, 5, 4, 8, 11, 3]
[9, 5, 10, 8, 3, 1, 12, 5, 4, 8, 11, 3]
[10, 7, 9, 4, 3, 12, 6, 8, 8, 8, 11, 3]
[10, 7, 9, 4, 3, 12, 6, 8, 8, 8, 11, 3]
[10, 7, 9, 4, 3, 12, 6, 8, 8, 8, 11, 3]
[2, 3, 9, 11, 5, 8, 11, 1, 8, 8, 11, 3]
[2, 3, 9, 11, 5, 8, 11, 1, 8, 8, 11, 3]
[2, 3, 9, 11, 5, 8, 11, 1, 8, 8, 11, 3]
[2, 3, 9, 11, 5, 8, 11, 1, 8, 8, 11, 3]
[2, 3, 9, 11, 5, 8, 11, 1, 8, 8, 11, 3]
[5, 1, 3, 2, 10, 9, 12, 8, 8, 8, 11, 3]
[5, 1, 3, 2, 10, 9, 12, 8, 8, 8, 11, 3]
[5, 1, 3, 2, 10, 9, 12, 8, 8, 8, 11, 3]
[10, 9, 3, 7, 5, 4, 1, 12, 11, 8, 11, 3]
[10, 9, 3, 7, 5, 4, 1, 12, 11, 8, 11, 3]
[2, 7, 9, 10, 12, 1, 6, 5, 8, 11, 1

[12, 4, 9, 10, 1, 3, 2, 5, 6, 8, 11, 3]
[6, 10, 7, 12, 4, 1, 9, 11, 8, 5, 11, 3]
[3, 6, 12, 9, 10, 2, 7, 5, 1, 8, 11, 3]
[1, 4, 12, 9, 10, 8, 7, 5, 1, 4, 6, 3]
[1, 4, 12, 9, 10, 8, 7, 5, 1, 4, 6, 3]
[5, 10, 7, 6, 1, 2, 8, 11, 11, 8, 11, 3]
[5, 10, 7, 6, 1, 2, 8, 11, 11, 8, 11, 3]
[5, 10, 7, 6, 1, 2, 8, 11, 11, 8, 11, 3]
[3, 6, 12, 2, 9, 10, 3, 5, 8, 11, 11, 3]
[3, 6, 12, 2, 9, 10, 3, 5, 8, 11, 11, 3]
[3, 6, 12, 2, 9, 10, 3, 5, 8, 11, 11, 3]
[9, 7, 1, 3, 2, 10, 8, 5, 8, 11, 11, 3]
[9, 7, 1, 3, 2, 10, 8, 5, 8, 11, 11, 3]
[9, 7, 1, 3, 2, 10, 8, 5, 8, 11, 11, 3]
[10, 9, 7, 5, 2, 8, 4, 11, 5, 12, 6, 1]
[10, 9, 7, 5, 2, 8, 1, 11, 5, 12, 6, 3]
[4, 10, 12, 2, 6, 9, 11, 8, 6, 4, 3, 1]
[4, 10, 12, 2, 6, 9, 11, 8, 6, 4, 3, 1]
[10, 4, 9, 6, 2, 1, 5, 8, 11, 11, 11, 3]
[10, 4, 9, 6, 2, 1, 5, 8, 11, 11, 11, 3]
[10, 7, 9, 6, 2, 1, 8, 12, 4, 8, 11, 3]
[10, 7, 9, 6, 4, 1, 12, 3, 8, 11, 11, 3]
[10, 7, 9, 6, 4, 1, 12, 3, 8, 11, 11, 3]
[1, 10, 3, 9, 5, 11, 8, 6, 4, 1, 7, 3]
[1, 10, 3, 9, 5, 11, 8, 6, 4, 1,

[5, 7, 12, 10, 2, 5, 9, 8, 11, 8, 11, 3]
[5, 7, 12, 10, 2, 5, 9, 8, 11, 8, 11, 3]
[5, 7, 12, 10, 2, 5, 9, 8, 11, 8, 11, 3]
[3, 10, 8, 9, 1, 2, 4, 6, 12, 11, 8, 3]
[3, 10, 8, 9, 1, 2, 4, 6, 12, 11, 8, 3]
[12, 6, 7, 4, 10, 9, 1, 2, 8, 11, 11, 3]
[10, 9, 2, 1, 7, 6, 4, 12, 11, 5, 7, 3]
[1, 7, 10, 9, 2, 11, 5, 12, 6, 4, 11, 3]
[6, 12, 5, 8, 11, 2, 7, 1, 3, 5, 8, 11]
[6, 12, 5, 8, 11, 2, 7, 1, 3, 5, 8, 11]
[6, 12, 5, 8, 11, 2, 7, 1, 3, 5, 8, 11]
[2, 10, 9, 7, 1, 6, 4, 12, 3, 5, 11, 3]
[3, 10, 8, 5, 12, 4, 1, 7, 5, 8, 11, 3]
[3, 10, 8, 5, 12, 4, 1, 7, 5, 8, 11, 3]
[3, 10, 8, 5, 12, 4, 1, 7, 5, 8, 11, 3]
[6, 7, 9, 1, 10, 12, 5, 8, 11, 8, 11, 3]
[6, 7, 9, 1, 10, 12, 5, 8, 11, 8, 11, 3]
[9, 4, 7, 10, 1, 2, 8, 11, 11, 8, 5, 3]
[9, 4, 7, 10, 1, 2, 8, 11, 11, 8, 5, 3]
[9, 10, 2, 7, 5, 4, 1, 12, 11, 8, 11, 3]
[6, 4, 9, 12, 11, 7, 1, 8, 5, 8, 11, 3]
[6, 4, 9, 12, 11, 7, 1, 8, 5, 8, 11, 3]
[9, 10, 2, 7, 5, 8, 2, 3, 6, 12, 3, 11]
[9, 10, 2, 7, 5, 8, 2, 3, 6, 12, 3, 11]
[4, 6, 8, 12, 5, 10, 10, 7, 11, 

[2, 7, 9, 3, 1, 5, 8, 11, 11, 8, 11, 3]
[2, 7, 9, 3, 1, 5, 8, 11, 11, 8, 11, 3]
[2, 7, 9, 3, 1, 5, 8, 11, 11, 8, 11, 3]
[2, 10, 6, 11, 9, 7, 12, 8, 11, 8, 11, 3]
[2, 10, 6, 11, 9, 7, 12, 8, 11, 8, 11, 3]
[2, 10, 6, 11, 9, 7, 12, 8, 11, 8, 11, 3]
[9, 10, 2, 5, 7, 8, 11, 1, 11, 8, 11, 3]
[9, 10, 2, 5, 7, 8, 11, 1, 11, 8, 11, 3]
[9, 10, 2, 5, 7, 8, 11, 1, 11, 8, 11, 3]
[11, 10, 7, 12, 6, 4, 1, 2, 8, 11, 3, 3]
[11, 10, 7, 12, 6, 4, 1, 2, 8, 11, 3, 3]
[9, 2, 5, 7, 12, 1, 7, 8, 11, 3, 11, 3]
[9, 2, 5, 7, 12, 1, 7, 8, 11, 3, 11, 3]
[9, 2, 5, 7, 12, 1, 7, 8, 11, 3, 11, 3]
[1, 6, 9, 10, 2, 5, 8, 11, 12, 8, 11, 3]
[1, 6, 9, 10, 2, 5, 8, 11, 12, 8, 11, 3]
[9, 2, 5, 3, 7, 12, 1, 7, 11, 8, 5, 3]
[9, 2, 5, 3, 7, 12, 1, 7, 11, 8, 5, 3]
[9, 2, 5, 3, 7, 12, 1, 7, 11, 8, 5, 3]
[10, 2, 9, 6, 7, 1, 4, 12, 6, 3, 8, 11]
[1, 10, 7, 9, 12, 4, 1, 3, 4, 6, 11, 3]
[1, 10, 7, 9, 12, 4, 1, 3, 4, 6, 11, 3]
[1, 10, 7, 9, 12, 4, 1, 3, 4, 6, 11, 3]
[12, 6, 4, 3, 5, 8, 10, 9, 11, 2, 3, 7]
[7, 1, 10, 6, 9, 2, 12, 4, 8, 

[4, 6, 12, 2, 10, 9, 11, 5, 3, 8, 11, 3]
[4, 6, 12, 2, 10, 9, 11, 5, 3, 8, 11, 3]
[10, 4, 1, 9, 7, 12, 5, 2, 11, 5, 11, 3]
[10, 4, 1, 9, 7, 12, 5, 2, 11, 5, 11, 3]
[2, 9, 10, 4, 8, 11, 5, 1, 8, 8, 11, 3]
[2, 9, 10, 4, 8, 11, 5, 1, 8, 8, 11, 3]
[2, 9, 10, 4, 8, 11, 5, 1, 8, 8, 11, 3]
[12, 9, 6, 4, 3, 2, 5, 8, 11, 8, 11, 3]
[12, 9, 6, 4, 3, 2, 5, 8, 11, 8, 11, 3]
[12, 9, 6, 4, 3, 2, 5, 8, 11, 8, 11, 3]
[3, 2, 6, 4, 12, 10, 9, 5, 11, 8, 11, 3]
[3, 2, 6, 4, 12, 10, 9, 5, 11, 8, 11, 3]
[7, 10, 6, 4, 12, 1, 7, 11, 2, 8, 11, 3]
[7, 10, 6, 4, 12, 1, 7, 11, 2, 8, 11, 3]
[10, 6, 2, 5, 12, 9, 2, 4, 8, 11, 5, 3]
[10, 6, 2, 5, 12, 9, 2, 4, 8, 11, 5, 3]
[9, 10, 7, 5, 8, 1, 11, 4, 5, 11, 11, 3]
[9, 10, 7, 5, 8, 1, 11, 4, 5, 11, 11, 3]
[9, 10, 7, 5, 8, 1, 11, 4, 5, 11, 11, 3]
[10, 6, 3, 7, 12, 9, 2, 1, 8, 11, 3, 5]
[12, 7, 9, 6, 1, 2, 10, 11, 5, 8, 11, 3]
[4, 10, 12, 9, 1, 6, 7, 11, 8, 8, 11, 3]
[4, 10, 12, 9, 1, 6, 7, 11, 8, 8, 11, 3]
[10, 8, 11, 2, 5, 1, 5, 8, 11, 8, 3, 3]
[10, 8, 11, 2, 5, 1, 5, 8,